In [1]:
import nltk
import sys 
import re
from sklearn.decomposition  import  PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from nltk.corpus import stopwords

In [2]:
train=pd.read_json("./train.zip")
test=pd.read_json("./test_questions.zip")

In [3]:
train=train[["id","text"]]
test=test[["id","text"]]

In [4]:
train

,id,text
0,1016267329137729536,"@Fhumu_Makhwanya Hi, no you cannot withdraw fu..."
1,1016270646060384256,"@GTalevi Hi, can you please share more informa..."
2,1016272104839434240,"@Mandilele_ Hi, thank you for sharing your exp..."
3,1016275249753743360,@GTalevi Please DM us your Home-Loan account n...
4,1016277410545889281,@Angvanreenen83 Please provide us with it.
...,...,...
96557,1030149995490697217,"@Dopla101 Hi, apologies for the delayed respon..."
96558,1030162359933906945,"@BlackRose__ZA Hi.\nKindly DM your query, affe..."
96559,1030170340473929730,"@aminah_bhengu Hi, please inbox us your ID &am..."
96560,1030172343937716224,"@reitumakwea Hi, please follow &amp; inbox us ..."


In [5]:
test

,id,text
0,1028546931453243392,@_Warith Hi Wari. please contact our Customer ...
1,1028548116297986050,"@ziyaad_seedat Hi there Ziyaad, the Samsung Pa..."
2,1028550157699350528,@180DegreesCC Lovely packaging. How long have ...
3,1028552399307399169,"@ConsolerM Hi there, please give us your conta..."
4,1028559460745150465,@zimcritic Loving yourself looks beautiful on ...
...,...,...
32205,1032883399898423298,"@lee_masilo Hi @lee_masilo, sorry for late res..."
32206,1032886868902334464,@BeckeChris Thanks Chris. Network informed. Ne...
32207,1032887467526049792,@ntandoz_nmz You're welcome
32208,1032888433788825601,"@sne_mthethwa Hi sne_mthethwa, we would like t..."


In [6]:
# Lower case
test['preprocessed'] = test['text'].str.lower()
train['preprocessed'] = train['text'].str.lower()

In [7]:
# Turning text columns to list
train_list = train['preprocessed'].tolist()
test_list = test['preprocessed'].tolist()

In [9]:
!pip3 install ekphrasis

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 80 kB 7.4 MB/s eta 0:00:011
     |████████████████████████████████| 53 kB 30.1 MB/s eta 0:00:01
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-py3-none-any.whl size=82843 sha256=6928cb3a1c75982ee9c2e33a6393f66f03746bba01a7052643b247129714f29e
  Stored in directory: /scratch/4164544.1.academic/pip-ephem-wheel-cache-e3piv36e/wheels/1e/36/c7/477ab8220745f8d1f84b56de0ab88544b410f80a830f6feee6
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=7c707cbf2450e3b0f28e524450f1d95da29fba7874f1ca7877d449f9978c4c2f
  Stored in directory: /scratch/4164544.1.academic/pip-ephem-wheel-cache-e3piv36e/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built ekphrasis termcolor
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider up

In [10]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [11]:
# Text preprocessing 
# from https://colab.research.google.com/drive/1Uoog-scrKhLX3QZmrs1ClxCFdGIMH88i?usp=sharing#scrollTo=SV2zCbTag8QR

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr4/cs505/ppaudel/.local/lib/python3.8/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /usr4/cs505/ppaudel/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /usr4/cs505/ppaudel/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr4/cs505/ppaudel/.local/lib/python3.8/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [12]:
# Applying text preprocessor
temp_list1 = []

for s in train_list:
    s = (" ".join(text_processor.pre_process_doc(s)))
    temp_list1.append(s)
    
temp_list2 = []

for s in test_list:
    s = (" ".join(text_processor.pre_process_doc(s)))
    temp_list2.append(s)

In [13]:
# Remove punctuation
import re

train_list2 = []
for sentence in temp_list1:
    sentence = re.sub(r"[^\w\d\s\<\>]+",'',sentence)
    train_list2.append(sentence)
    
test_list2 = []
for sentence in temp_list2:
    sentence = re.sub(r"[^\w\d\s\<\>]+",'',sentence)
    test_list2.append(sentence)

In [14]:
# List back to df
train['preprocessed'] = train_list2
test['preprocessed'] = test_list2

In [15]:
# Remove stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')
test['preprocessed'] = test['preprocessed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
train['preprocessed'] = train['preprocessed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#Remove underscore
train['preprocessed'] = train['preprocessed'].replace('_', '', regex=True)
test['preprocessed'] = test['preprocessed'].replace('_', '', regex=True)

In [16]:
# Tokenize
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
train['tokenized']=train['preprocessed'].apply(tt.tokenize)
test['tokenized']=test['preprocessed'].apply(tt.tokenize)

In [8]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def return_sentiments(idx,text):
    try:
        sid_obj = SentimentIntensityAnalyzer()
        # polarity_scores method of SentimentIntensityAnalyzer
        # object gives a sentiment dictionary.
        # which contains pos, neg, neu, and compound scores.
        sentiment_dict = sid_obj.polarity_scores(text)
        return (idx,sentiment_dict['neg'],sentiment_dict['neu'],sentiment_dict['pos'],sentiment_dict['compound'])
    except Exception as e:
        return(idx,-10,-10,-10,-10,-10)

In [9]:
from tqdm import tqdm
tqdm.pandas()
train_vader=train[['id','text']].progress_apply(lambda x:return_sentiments(x['id'],x['text']),axis=1)


100%|██████████| 96562/96562 [18:24<00:00, 87.43it/s] 


In [10]:
from tqdm import tqdm
tqdm.pandas()
test_vader=test[['id','text']].progress_apply(lambda x:return_sentiments(x['id'],x['text']),axis=1)


100%|██████████| 32210/32210 [05:45<00:00, 93.22it/s] 


In [13]:
test_vader[0]

(1028546931453243392, 0.13, 0.628, 0.242, 0.3182)

In [18]:
xx=open("train_vader_sentiment.csv","w")
for tup in train_vader:
    xx.write("%d"%tup[0])
    xx.write(",")
    xx.write("%f"%tup[1])#neg
    xx.write(",")
    xx.write("%f"%tup[2]) #neu
    xx.write(",")
    xx.write("%f"%tup[3]) #pos
    xx.write(",")
    xx.write("%f"%tup[4]) #compound
    xx.write("\n")
xx.close()

In [19]:
xx=open("test_vader_sentiment.csv","w")
for tup in test_vader:
    xx.write("%d"%tup[0])
    xx.write(",")
    xx.write("%f"%tup[1])#neg
    xx.write(",")
    xx.write("%f"%tup[2]) #neu
    xx.write(",")
    xx.write("%f"%tup[3]) #pos
    xx.write(",")
    xx.write("%f"%tup[4]) #compound
    xx.write("\n")
xx.close()

In [58]:
stop = stopwords.words('english')
test['text_without_stopwords'] = test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
train['text_without_stopwords'] = train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [59]:
test['tokenized_text'] = test.apply(lambda row: nltk.word_tokenize(row['text_without_stopwords']), axis=1)
train['tokenized_text'] = train.apply(lambda row: nltk.word_tokenize(row['text_without_stopwords']), axis=1)

In [60]:
train

,id,text,text_without_stopwords,tokenized_text
0,1016267329137729536,fhumumakhwanya hi no you cannot withdraw funds...,fhumumakhwanya hi cannot withdraw funds transf...,"[fhumumakhwanya, hi, can, not, withdraw, funds..."
1,1016270646060384256,gtalevi hi can you please share more informati...,gtalevi hi please share information regarding,"[gtalevi, hi, please, share, information, rega..."
2,1016272104839434240,mandilele hi thank you for sharing your experi...,mandilele hi thank sharing experience us,"[mandilele, hi, thank, sharing, experience, us]"
3,1016275249753743360,gtalevi please dm us your homeloan account num...,gtalevi please dm us homeloan account number c...,"[gtalevi, please, dm, us, homeloan, account, n..."
4,1016277410545889281,angvanreenen please provide us with it,angvanreenen please provide us,"[angvanreenen, please, provide, us]"
...,...,...,...,...
96557,1030149995490697217,dopla hi apologies for the delayed response pl...,dopla hi apologies delayed response please ref...,"[dopla, hi, apologies, delayed, response, plea..."
96558,1030162359933906945,blackroseza hi kindly dm your query affected p...,blackroseza hi kindly dm query affected phone ...,"[blackroseza, hi, kindly, dm, query, affected,..."
96559,1030170340473929730,aminahbhengu hi please inbox us your id and or...,aminahbhengu hi please inbox us id order numbe...,"[aminahbhengu, hi, please, inbox, us, id, orde..."
96560,1030172343937716224,reitumakwea hi please follow and inbox us your...,reitumakwea hi please follow inbox us query af...,"[reitumakwea, hi, please, follow, inbox, us, q..."


In [61]:
test

,id,text,text_without_stopwords,tokenized_text
0,1028546931453243392,warith hi wari please contact our customer com...,warith hi wari please contact customer complai...,"[warith, hi, wari, please, contact, customer, ..."
1,1028548116297986050,ziyaadseedat hi there ziyaad the samsung pay d...,ziyaadseedat hi ziyaad samsung pay also work v...,"[ziyaadseedat, hi, ziyaad, samsung, pay, also,..."
2,1028550157699350528,degreescc lovely packaging how long have you b...,degreescc lovely packaging long catering confe...,"[degreescc, lovely, packaging, long, catering,..."
3,1028552399307399169,consolerm hi there please give us your contact...,consolerm hi please give us contact details as...,"[consolerm, hi, please, give, us, contact, det..."
4,1028559460745150465,zimcritic loving yourself looks beautiful on y...,zimcritic loving looks beautiful tell us makes...,"[zimcritic, loving, looks, beautiful, tell, us..."
...,...,...,...,...
32205,1032883399898423298,leemasilo hi leemasilo sorry for late response...,leemasilo hi leemasilo sorry late response ple...,"[leemasilo, hi, leemasilo, sorry, late, respon..."
32206,1032886868902334464,beckechris thanks chris network informed netwo...,beckechris thanks chris network informed netwo...,"[beckechris, thanks, chris, network, informed,..."
32207,1032887467526049792,ntandoznmz youre welcome,ntandoznmz youre welcome,"[ntandoznmz, youre, welcome]"
32208,1032888433788825601,snemthethwa hi snemthethwa we would like to as...,snemthethwa hi snemthethwa would like assist m...,"[snemthethwa, hi, snemthethwa, would, like, as..."


In [11]:
# Work on Glove , and Pretrained Doc2Vec

In [12]:
import gensim.models as g

In [13]:
import logging

#doc2vec parameters
vector_size = 300
window_size = 15
min_count = 1
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 0 #0 = dbow; 1 = dmpv
worker_count = 4 #number of parallel processes


In [19]:
train

,id,text,preprocessed,tokenized
0,1016267329137729536,"@Fhumu_Makhwanya Hi, no you cannot withdraw fu...",<user> hi cannot withdraw funds transfer pures...,"[<user>, hi, cannot, withdraw, funds, transfer..."
1,1016270646060384256,"@GTalevi Hi, can you please share more informa...",<user> hi please share information regarding,"[<user>, hi, please, share, information, regar..."
2,1016272104839434240,"@Mandilele_ Hi, thank you for sharing your exp...",<user> hi thank sharing experience us,"[<user>, hi, thank, sharing, experience, us]"
3,1016275249753743360,@GTalevi Please DM us your Home-Loan account n...,<user> please dm us home loan account number c...,"[<user>, please, dm, us, home, loan, account, ..."
4,1016277410545889281,@Angvanreenen83 Please provide us with it.,<user> please provide us,"[<user>, please, provide, us]"
...,...,...,...,...
96557,1030149995490697217,"@Dopla101 Hi, apologies for the delayed respon...",<user> hi apologies delayed response please re...,"[<user>, hi, apologies, delayed, response, ple..."
96558,1030162359933906945,"@BlackRose__ZA Hi.\nKindly DM your query, affe...",<user> hi kindly dm query affected phone numbe...,"[<user>, hi, kindly, dm, query, affected, phon..."
96559,1030170340473929730,"@aminah_bhengu Hi, please inbox us your ID &am...",<user> hi please inbox us id order number cont...,"[<user>, hi, please, inbox, us, id, order, num..."
96560,1030172343937716224,"@reitumakwea Hi, please follow &amp; inbox us ...",<user> hi please follow inbox us query affecte...,"[<user>, hi, please, follow, inbox, us, query,..."


In [20]:
test

,id,text,preprocessed,tokenized
0,1028546931453243392,@_Warith Hi Wari. please contact our Customer ...,<user> hi wari please contact customer complai...,"[<user>, hi, wari, please, contact, customer, ..."
1,1028548116297986050,"@ziyaad_seedat Hi there Ziyaad, the Samsung Pa...",<user> hi ziyaad samsung pay also work visa ca...,"[<user>, hi, ziyaad, samsung, pay, also, work,..."
2,1028550157699350528,@180DegreesCC Lovely packaging. How long have ...,<user> lovely packaging long catering confecti...,"[<user>, lovely, packaging, long, catering, co..."
3,1028552399307399169,"@ConsolerM Hi there, please give us your conta...",<user> hi please give us contact details assis...,"[<user>, hi, please, give, us, contact, detail..."
4,1028559460745150465,@zimcritic Loving yourself looks beautiful on ...,<user> loving looks beautiful tell us makes pr...,"[<user>, loving, looks, beautiful, tell, us, m..."
...,...,...,...,...
32205,1032883399898423298,"@lee_masilo Hi @lee_masilo, sorry for late res...",<user> hi <user> sorry late response please dm...,"[<user>, hi, <user>, sorry, late, response, pl..."
32206,1032886868902334464,@BeckeChris Thanks Chris. Network informed. Ne...,<user> thanks chris network informed network t...,"[<user>, thanks, chris, network, informed, net..."
32207,1032887467526049792,@ntandoz_nmz You're welcome,<user> welcome,"[<user>, welcome]"
32208,1032888433788825601,"@sne_mthethwa Hi sne_mthethwa, we would like t...",<user> hi snemthethwa would like assist may pl...,"[<user>, hi, snemthethwa, would, like, assist,..."


In [22]:
train_texts=train['tokenized'].to_list()
test_texts=test['tokenized'].to_list()

In [65]:
len(vocabulary_set)

120416

In [24]:
dict_w2v = {} 
import numpy as np
with open('./HW4/glove.twitter.27B.100d.txt', "r") as file:     
    for line in file:         
       tokens = line.split()      
       word = tokens[0]         
       vector = np.array(tokens[1:], dtype=np.float32) 
       if vector.shape[0] == 100:    
           dict_w2v[word] = vector 
       else:      
           print("There was an issue with " + word) 
# let's check the vocabulary size 
print("Dictionary Size: ", len(dict_w2v)) 

There was an issue with -0.32053
Dictionary Size:  1193513


In [26]:
train_vecs={}
test_vecs={}
empty=0
for index, row in train.iterrows():
    text=row['tokenized']
    idx=row['id']
    vec=[]
    for token in text:
        if token in dict_w2v:
            vec.append(dict_w2v[token])
    if len(vec)==0:
        empty+=1
        vec=[0]*100 # Or some better initialization ?
    else:
        vec=np.mean(vec,axis=0)
    train_vecs[idx]=vec
        

In [27]:
print(empty," Tweets with 0 vocab hit")

2  Tweets with 0 vocab hit


In [28]:
empty=0
for index, row in test.iterrows():
    text=row['tokenized']
    idx=row['id']
    vec=[]
    for token in text:
        if token in dict_w2v:
            vec.append(dict_w2v[token])
    if len(vec)==0:
        empty+=1
        vec=[0]*100 # Or some better initialization ?
    else:
        vec=np.mean(vec,axis=0)
    test_vecs[idx]=vec

In [29]:
print(empty," Tweets with 0 vocab hit")

0  Tweets with 0 vocab hit


In [30]:
len(train_texts),len(train_vecs)

(96562, 96562)

In [31]:
train_full=pd.read_json("./train.zip")
test_full=pd.read_json("./test_questions.zip")

In [32]:
train_full.head()

,created_at,id,id_str,text,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,favorite_count,favorited,retweeted,lang,possibly_sensitive,extended_entities,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status
0,2018-07-09 10:26:27+00:00,1016267329137729536,1016267329137729536,"@Fhumu_Makhwanya Hi, no you cannot withdraw fu...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://www.lithium.com"" rel=""nofollo...",1.016267e+18,1.016267e+18,7.232664e+07,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-07-09 10:39:38+00:00,1016270646060384256,1016270646060384256,"@GTalevi Hi, can you please share more informa...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://www.lithium.com"" rel=""nofollo...",1.016270e+18,1.016270e+18,3.701130e+08,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-07-09 10:45:25+00:00,1016272104839434240,1016272104839434240,"@Mandilele_ Hi, thank you for sharing your exp...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://www.lithium.com"" rel=""nofollo...",1.016272e+18,1.016272e+18,2.770973e+09,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-07-09 10:57:55+00:00,1016275249753743360,1016275249753743360,@GTalevi Please DM us your Home-Loan account n...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://www.lithium.com"" rel=""nofollo...",1.016273e+18,1.016273e+18,3.701130e+08,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-07-09 11:06:30+00:00,1016277410545889281,1016277410545889280,@Angvanreenen83 Please provide us with it.,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://www.lithium.com"" rel=""nofollo...",1.016277e+18,1.016277e+18,4.149807e+08,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
test_full.head()

,created_at,id,id_str,text,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,is_quote_status,favorited,retweeted,lang,possibly_sensitive,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,extended_entities
0,2018-08-12 07:41:12+00:00,1028546931453243392,1028546931453243392,@_Warith Hi Wari. please contact our Customer ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://www.lithium.com"" rel=""nofollo...",1.028226e+18,1.028226e+18,1.868096e+08,...,False,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-08-12 07:45:55+00:00,1028548116297986050,1028548116297986048,"@ziyaad_seedat Hi there Ziyaad, the Samsung Pa...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://www.lithium.com"" rel=""nofollo...",1.028206e+18,1.028206e+18,1.028205e+18,...,False,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-08-12 07:54:01+00:00,1028550157699350528,1028550157699350528,@180DegreesCC Lovely packaging. How long have ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://www.lithium.com"" rel=""nofollo...",1.028461e+18,1.028461e+18,1.389484e+09,...,False,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-08-12 08:02:56+00:00,1028552399307399169,1028552399307399168,"@ConsolerM Hi there, please give us your conta...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://www.lithium.com"" rel=""nofollo...",1.028516e+18,1.028516e+18,3.198895e+08,...,False,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-08-12 08:30:59+00:00,1028559460745150465,1028559460745150464,@zimcritic Loving yourself looks beautiful on ...,False,"{'hashtags': [{'text': 'WomenISeeYou', 'indice...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",1.028272e+18,1.028272e+18,7.331674e+08,...,False,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
ff=open("train_ids_glove","w")
for key in train_vecs:
    ff.write("%d"%key)
    ff.write(",")
    for item in train_vecs[key]:
        ff.write("%f"%item)
        ff.write(",")
    ff.write("\n")
ff.close()

In [41]:
ff=open("test_ids_glove","w")
for key in test_vecs:
    ff.write("%d"%key)
    ff.write(",")
    for item in test_vecs[key]:
        ff.write("%f"%item)
        ff.write(",")
    ff.write("\n")
ff.close()

In [72]:
df_train=pd.read_csv("train_custom_features.csv")

In [73]:
df_test=pd.read_csv("test_custom_features.csv")

In [74]:
df_train_clean=df_train[['year','month','week_day','hour','is_reply_to_tweet',
                        'is_reply_to_user','@_count','#_count','url_count',
                        'tweet_len']]

In [75]:
final_train_vecs=[]
for index, row in df_train.iterrows():
    idx=row['tweet_id']
    final_train_vecs.append(train_vecs[idx])

In [76]:
final_test_vecs=[]
for index, row in df_test.iterrows():
    idx=row['tweet_id']
    final_test_vecs.append(test_vecs[idx])

In [77]:
from sklearn.compose import ColumnTransformer
# construct the column transfomer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
column_transformer = ColumnTransformer(
    [('year', "passthrough", ['year']), 
    ('month', "passthrough", ['month']),
     ('week_day',"passthrough",['week_day']),
     ('hour',"passthrough",['hour']),
     ('is_reply_to_tweet',"passthrough",['is_reply_to_tweet']),
     ('is_reply_to_user',"passthrough",['is_reply_to_user']),
     ('@_count',"passthrough",['@_count']),
     ('#_count',"passthrough",['#_count']),
     ('url_count',"passthrough",['url_count']),
     ('tweet_len',"passthrough",['tweet_len']),
     ('glove_embedding',final_train_vecs,'glove_embedding'),
    ],
    remainder='drop')

In [78]:
import lightgbm as lgb


In [79]:
train_labels=df_train['rewteet_count'].to_list()

In [80]:
from sklearn.linear_model import LinearRegression

In [81]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
#model = lgb.LGBMClassifier(learning_rate=0.09,max_depth=5,random_state=42,verbose=5)
model=LinearRegression()
pipe = Pipeline([
                  ('transformer', column_transformer),
                  ('classify', model)
                ])

In [ ]:
pipe.fit(df_train,train_labels)

In [82]:
import pickle

## let o be the object whose size you want to measure
size_estimate = len(pickle.dumps(df_train))

In [83]:
size_estimate

14678485

In [84]:
import sys
print(sys.getsizeof(df_train))

14677568
